<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [10]:
(x, y), (_, _) = tf.keras.datasets.mnist.load_data()

In [11]:
x = np.expand_dims(x, axis=-1)
x = (x - 127.5) / 127.5
x.shape, x.min(), x.max()

((60000, 28, 28, 1), -1.0, 1.0)

In [12]:
buffer_size=x.shape[0]
batch_size=256

In [13]:
x = tf.data.Dataset.from_tensor_slices(x).shuffle(buffer_size).batch(batch_size)

In [14]:
x

<BatchDataset element_spec=TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float64, name=None)>

In [16]:
def build_generator():
  network = tf.keras.Sequential()

  network.add(tf.keras.layers.Dense(7*7*256, use_bias=False, input_shape=(100, )))
  network.add(tf.keras.layers.BatchNormalization())
  network.add(tf.keras.layers.LeakyReLU())


  network.add(tf.keras.layers.Reshape(7, 7, 256))

  network.add(tf.keras.layers.Conv2DTranspose(128, 5, padding='same', use_bias=False))
  network.add(tf.keras.layers.BatchNormalization())
  network.add(tf.keras.layers.LeakyReLU())


  network.add(tf.keras.layers.Conv2DTranspose(64, 5, strides=2, padding='same', use_bias=False))
  network.add(tf.keras.layers.BatchNormalization())
  network.add(tf.keras.layers.LeakyReLU())


  network.add(tf.keras.layers.Conv2DTranspose(1, 5, strides=2, padding='same', use_bias=False, activation='tanh'))

  return network